In [1]:
from aicspylibczi import CziFile
import czifile
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import cv2
import os
import ffmpeg
import time
import pandas as pd
from cellpose import io, models

In [34]:
def get_file(vid_type, num):
    root_dir = "/mnt/datadisk/"
    vid_type = vid_type.lower()
    if(vid_type == "processed"):
        folder = 'FactinProcessed'
        fname = f'dicty_factin_pip3-0{num}_processed.czi'
    elif(vid_type == 'mip'):
        folder = 'FactinMIP'
        fname = f'dicty_factin_pip3-0{num}_MIP.czi'
    elif(vid_type == 'new'):
        folder = 'FactinMIP'
        fname = f'New-0{num}_MIP.czi'
    else:
        print(f"invalid type, try again from {['processed', 'mip', 'new']}")
        return(-1)
    fpath = os.path.join(root_dir, folder, fname)
    video = CziFile(fpath)
    print(f"Loading {fname} with dims {video.get_dims_shape()}")
    return(video)

In [30]:
video = get_file('mip', 3)

Loading dicty_factin_pip3-03_MIP.czi with dims [{'X': (0, 474), 'Y': (0, 2048), 'C': (0, 2), 'T': (0, 90)}]


## Helper funcs

In [35]:
def get_image(timestep=-1, z_plane=50, video=video, channel=0):
    dims = video.get_dims_shape()[0]
    
    timestep = timestep % dims['T'][-1]
    z_plane = z_plane % dims['Z'][-1]
    img, shp = video.read_image(S=0, Z=z_plane, T=timestep, C=channel)
    #print(shp)
    return(img.squeeze()) # returns (channels, Y, X)

In [36]:
def scale_img(img):
        lower_bound = np.percentile(img, 1)
        upper_bound = np.percentile(img, 99.9)
        I = (img - lower_bound) / (upper_bound - lower_bound)
        I = np.clip(I, 0, 1)
        return(I)

def get_plane(z_plane, video, apply_scaling=True, channel=0):
    plane, shp = video.read_image(S=0, Z=z_plane, C=channel)
    if(apply_scaling):
        plane = scale_img(plane)

    return(plane.squeeze())

In [5]:
def get_bounding_box(mask, cellID, padding=3):
    binary_array = (mask == cellID)
    # Convert binary array to binary image
    binary_image = np.uint8(binary_array * 255)
    
    # Find contours
    contours, _ = cv2.findContours(binary_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if(len(contours) == 0):
        print("Contour not found")
        return(-1)
    # Get the bounding rectangle
    x, y, w, h = cv2.boundingRect(contours[0])
    
    # Apply padding
    x -= padding
    y -= padding
    w += 2 * padding
    h += 2 * padding

    return(x, y, w, h)

In [6]:
def track_cell(cell_id, frames=frames, masks=masks, padding=2):
    data = {"patches" : [], 'boxes' : [], "masks" : []}
    num_frames = frames.shape[0]
    h_max, w_max = 0, 0
    skip_frames = []
    for i in range(num_frames):
        box = get_bounding_box(masks[i], cell_id, padding)
        if(box != -1): # valid box
            x, y, w, h = box
            if(h > h_max):
                h_max = h
            if(w > w_max):
                w_max = w
            data['boxes'].append((x, y, w, h))
        else:
            skip_frames.append(i)
    print(f"Skipped frames {skip_frames}")
    for i, box in enumerate(data['boxes']):
        if i not in skip_frames: 
            x, y, w, h = box
            patch = frames[i, y:y+h_max, x:x+w_max]
            mask_patch = masks[i, y:y+h_max, x:x+w_max]
            data['patches'].append(patch)
            data['masks'].append(mask_patch)

    return(data)

NameError: name 'frames' is not defined

### Trying Johannes Method

In [17]:
frames = get_plane(50, video)